In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# df_weather_train = pd.read_csv("../ashrae-energy-prediction/train.csv")
df_weather_train = pd.read_csv("train.csv")
df_weather_train.info()

In [ ]:
# Load the dataset (assuming you've already read it)
df_weather_train.describe()

In [ ]:
df_weather_train.isnull().sum()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
sns.histplot(df_weather_train["meter_reading"], bins=50, kde=True)
plt.xlabel("Meter Reading")
plt.ylabel("Frequency")
plt.title("Distribution of Meter Readings")
plt.show()

In [ ]:
df_weather_train["meter"].value_counts().plot(kind="bar", title="Meter Type Distribution")
plt.xlabel("Meter Type")
plt.ylabel("Count")
plt.show()

In [ ]:
df_weather_train["timestamp"] = pd.to_datetime(df_weather_train["timestamp"])
df_weather_train["hour"] = df_weather_train["timestamp"].dt.hour
df_weather_train["day"] = df_weather_train["timestamp"].dt.day
df_weather_train["month"] = df_weather_train["timestamp"].dt.month
df_weather_train["year"] = df_weather_train["timestamp"].dt.year

In [ ]:
print(df_weather_train.describe())

In [ ]:
# Distribution of meter readings
sns.histplot(df_weather_train['meter_reading'], bins=50, kde=True)
plt.title("Distribution of Meter Readings")
plt.show()

# Trend analysis
plt.figure(figsize=(12, 5))
sns.lineplot(x=df_weather_train['timestamp'], y=df_weather_train['meter_reading'])
plt.title("Meter Readings Over Time")
plt.xticks(rotation=45)
plt.show()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define Generator Network
class Generator(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, output_dim)
        )

    def forward(self, x):
        return self.model(x)

# Define Discriminator Network
class Discriminator(nn.Module):
    def __init__(self, input_dim):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

# Training the GAN
def train_gan(real_data, epochs=1000):
    input_dim = real_data.shape[1]
    generator = Generator(input_dim, input_dim)
    discriminator = Discriminator(input_dim)
    optimizer_G = optim.Adam(generator.parameters(), lr=0.001)
    optimizer_D = optim.Adam(discriminator.parameters(), lr=0.001)
    loss_function = nn.BCELoss()

    for epoch in range(epochs):
        real_labels = torch.ones(real_data.shape[0], 1)
        fake_labels = torch.zeros(real_data.shape[0], 1)

        # Train Discriminator
        optimizer_D.zero_grad()
        real_predictions = discriminator(real_data)
        real_loss = loss_function(real_predictions, real_labels)

        noise = torch.randn(real_data.shape[0], input_dim)
        fake_data = generator(noise)
        fake_predictions = discriminator(fake_data.detach())
        fake_loss = loss_function(fake_predictions, fake_labels)

        d_loss = real_loss + fake_loss
        d_loss.backward()
        optimizer_D.step()

        # Train Generator
        optimizer_G.zero_grad()
        fake_predictions = discriminator(fake_data)
        g_loss = loss_function(fake_predictions, real_labels)
        g_loss.backward()
        optimizer_G.step()

        if epoch % 100 == 0:
            print(f"Epoch {epoch}, D Loss: {d_loss.item()}, G Loss: {g_loss.item()}")

    return generator

# Convert dataset to tensor and train GAN
real_data_tensor = torch.FloatTensor(df_weather_train[['meter_reading']].values)
real_data_tensor = real_data_tensor[:10000]
generator = train_gan(real_data_tensor, epochs=1000)

# Generate synthetic data
noise = torch.randn(10000, 1)
synthetic_meter_readings = generator(noise).detach().numpy()

print(synthetic_meter_readings[:100])